In [1]:
import pandas as pd #reading of the data set 
import numpy as np #matrix multiplication and data manipulation 
import matplotlib.pyplot as plt
import seaborn as sns


##iterate dataset into two parts one for training and other for testing 
from sklearn.model_selection import train_test_split,cross_val_score

##standardscaler function to standardize the numerical_cols in unit variance

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error,accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
df =pd.read_csv(r"C:\Users\pande\OneDrive\Desktop\AI_ML_PROJECT\heart-disease.csv.xls")

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
df=df.rename(columns={"target":"HeartDisease"})

In [5]:
df=pd.get_dummies(df,columns=["cp","restecg"])

In [6]:
df.head()

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,slope,ca,thal,HeartDisease,cp_0,cp_1,cp_2,cp_3,restecg_0,restecg_1,restecg_2
0,63,1,145,233,1,150,0,2.3,0,0,1,1,0,0,0,1,1,0,0
1,37,1,130,250,0,187,0,3.5,0,0,2,1,0,0,1,0,0,1,0
2,41,0,130,204,0,172,0,1.4,2,0,2,1,0,1,0,0,1,0,0
3,56,1,120,236,0,178,0,0.8,2,0,2,1,0,1,0,0,0,1,0
4,57,0,120,354,0,163,1,0.6,2,0,2,1,1,0,0,0,0,1,0


In [7]:
numerical_cols=["age","trestbps","chol","thalach","oldpeak"]
cat_cols=list(set(df.columns)-set(numerical_cols)-{"HeartDisease"})

In [8]:
cat_cols

['cp_1',
 'exang',
 'restecg_2',
 'slope',
 'restecg_1',
 'sex',
 'ca',
 'cp_2',
 'restecg_0',
 'cp_0',
 'cp_3',
 'fbs',
 'thal']

In [9]:
numerical_cols

['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Spliting and Training of Dataset.

In [10]:
df_train , df_test= train_test_split(df,test_size=0.2,random_state=42)

In [11]:
len(df_train),len(df_test)

(242, 61)

In [12]:
##creating object of standard scaler
scaler=StandardScaler()

#for custom input
def get_features_and_target_array(df,numericals_cols,cat_cols,scaler):
    x_numeric_scaled=scaler.fit_transform(df[numerical_cols])   ##normalization (0,1) scale
    x_categorical=df[cat_cols].to_numpy()   ## categorical cols
    x=np.hstack((x_categorical,x_numeric_scaled))

    return x

def get_features_and_target_arrays(df,numericals_cols,cat_cols,scaler):
    x_numeric_scaled=scaler.fit_transform(df[numerical_cols])   ##normalization (0,1) scale
    x_categorical=df[cat_cols].to_numpy()   ## categorical cols
    x=np.hstack((x_categorical,x_numeric_scaled))
    y=df["HeartDisease"]
    
    return x,y

x_train, y_train=get_features_and_target_arrays(df_train,cat_cols,numerical_cols,scaler)
X = df.drop("HeartDisease", axis=1)

y = df["HeartDisease"]

In [13]:
X

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,slope,ca,thal,cp_0,cp_1,cp_2,cp_3,restecg_0,restecg_1,restecg_2
0,63,1,145,233,1,150,0,2.3,0,0,1,0,0,0,1,1,0,0
1,37,1,130,250,0,187,0,3.5,0,0,2,0,0,1,0,0,1,0
2,41,0,130,204,0,172,0,1.4,2,0,2,0,1,0,0,1,0,0
3,56,1,120,236,0,178,0,0.8,2,0,2,0,1,0,0,0,1,0
4,57,0,120,354,0,163,1,0.6,2,0,2,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,140,241,0,123,1,0.2,1,0,3,1,0,0,0,0,1,0
299,45,1,110,264,0,132,0,1.2,1,0,3,0,0,0,1,0,1,0
300,68,1,144,193,1,141,0,3.4,1,2,3,1,0,0,0,0,1,0
301,57,1,130,131,0,115,1,1.2,1,1,3,1,0,0,0,0,1,0


## Logistic Regression Model 

In [14]:
## Training of the model 

clf=LogisticRegression()
clf.fit(x_train,y_train)
x_test,y_test=get_features_and_target_arrays(df_test,cat_cols,numerical_cols,scaler)
test_pred=clf.predict(x_test)
print("Error:",mean_squared_error(y_test,test_pred))
print("Accuracy:",accuracy_score(y_test,test_pred))

Error: 0.21311475409836064
Accuracy: 0.7868852459016393


In [15]:
test_pred

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0], dtype=int64)

In [16]:
# Prompt the user for input
age = int(input("Enter age: "))
sex = int(input("Enter sex (0 for female, 1 for male): "))
cp = int(input("Enter cp (chest pain type): "))
trestbps = int(input("Enter trestbps (resting blood pressure): "))
chol = int(input("Enter chol (serum cholesterol level): "))
fbs = int(input("Enter fbs (fasting blood sugar > 120 mg/dl): "))
restecg = int(input("Enter restecg (resting electrocardiographic results): "))
thalach = int(input("Enter thalach (maximum heart rate achieved): "))
exang = int(input("Enter exang (exercise induced angina): "))
oldpeak = float(input("Enter oldpeak (ST depression induced by exercise relative to rest): "))
slope = int(input("Enter slope (the slope of the peak exercise ST segment): "))
ca = int(input("Enter ca (number of major vessels (0-3) colored by fluoroscopy): "))
thal = int(input("Enter thal (thalassemia): "))

# Create a dictionary to store the input values
input_dict = {
    "age": age,
    "sex": sex,
    'cp_0': [int(cp == 0)],
    'cp_1': [int(cp == 1)],
    'cp_2': [int(cp == 2)],
    'cp_3': [int(cp == 3)],
    'restecg_0': [int(restecg == 0)],
    'restecg_1': [int(restecg == 1)],
    'restecg_2': [int(restecg == 2)],
    "trestbps": trestbps,
    "chol": chol,
    "fbs": fbs,
    "thalach": thalach,
    "exang": exang,
    "oldpeak": oldpeak,
    "slope": slope,
    "ca": ca,
    "thal": thal
}

# Convert the dictionary to a DataFrame
input_df = pd.DataFrame([input_dict])

# Use the get_features_and_target_arrays function to preprocess the input data
x_input= get_features_and_target_array(input_df, cat_cols, numerical_cols, scaler)

# Make predictions using the trained model
y_pred = clf.predict(x_input)

# Display the predicted heart disease status
if y_pred[0] == 0:
    print("Predicted Heart Disease Status: No")
else:
    print("Predicted Heart Disease Status: Yes")

Enter age: 37
Enter sex (0 for female, 1 for male): 1
Enter cp (chest pain type): 2
Enter trestbps (resting blood pressure): 130
Enter chol (serum cholesterol level): 250
Enter fbs (fasting blood sugar > 120 mg/dl): 0
Enter restecg (resting electrocardiographic results): 1
Enter thalach (maximum heart rate achieved): 187
Enter exang (exercise induced angina): 0
Enter oldpeak (ST depression induced by exercise relative to rest): 3.5
Enter slope (the slope of the peak exercise ST segment): 0
Enter ca (number of major vessels (0-3) colored by fluoroscopy): 0
Enter thal (thalassemia): 2
Predicted Heart Disease Status: Yes


In [18]:
confusion_matrix(y_test,test_pred)

array([[21,  8],
       [ 5, 27]], dtype=int64)